##Week4 Assignment
: text generation using LSTM

- 뉴스 파일은 kaggle or 사이언스팀 구글 드라이브에서 다운받으시면 됩니다

- text generation using RNN.ipynb파일과 https://wikidocs.net/45101를 참고하세요!

- 혹시 다른 파일로 해보고 싶으신 분들은 자유롭게 하시면 됩니다..!

- 4/9(금) 23:59 까지 '18기_000_week4.ipynb' 파일명으로 업로드 해주세요!

1) 데이터 이해/전처리

In [3]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [4]:
# ArticlesApril2018.csv 파일 로드
df=pd.read_csv('ArticlesApril2018.csv')   # 경로 설정
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [5]:
print('Number of columns: ', len(df.columns))
print(df.columns)

Number of columns:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [6]:
df['headline'].isnull().values.any()

False

In [7]:
# 모든 기사의 제목 하나의 리스트로 저장
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [8]:
# headline 중 unknown 처리

print('총 샘플의 개수 : {}'.format(len(headline))) # 현재 샘플의 개수
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

총 샘플의 개수 : 1324
노이즈값 제거 후 샘플의 개수 : 1214


In [9]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [12]:
# 데이터 전처리: 구두점 제거, lowercase()

########### fill in!  ############
import re
new_headline = []
for h in headline:
  h = h.lower()
  h = re.sub('[^a-zA-Z]+', ' ', h)
  new_headline.append(h)

new_headline[:5]

['former n f l cheerleaders settlement offer and a meeting with goodell',
 'e p a to unveil a new rule its effect less science in policymaking ',
 'the new noma explained',
 'how a bag of texas dirt became a times tradition',
 'is school a place for self expression ']

In [13]:
# 토큰화 및 단어 집합 크기 확인

########### fill in!  ############

tokenizer = Tokenizer()
tokenizer.fit_on_texts(new_headline)
vocab_size = len(tokenizer.word_index) + 1
print(f'단어 집합의 크기: {vocab_size}')

단어 집합의 크기: 3354


In [16]:
# 정수 인코딩 & 하나의 문장을 2 이상 길이의 훈련 데이터로 만들기"

########### fill in!  ############
sequences = []
for text in new_headline:
  for line in text.split('\n'):    # \n 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]   # 길이가 2 이상인 gram들 저장
        sequences.append(sequence)

print(sequences)


[[108, 124], [108, 124, 92], [108, 124, 92, 144], [108, 124, 92, 144, 387], [108, 124, 92, 144, 387, 1118], [108, 124, 92, 144, 387, 1118, 600], [108, 124, 92, 144, 387, 1118, 600, 8], [108, 124, 92, 144, 387, 1118, 600, 8, 2], [108, 124, 92, 144, 387, 1118, 600, 8, 2, 388], [108, 124, 92, 144, 387, 1118, 600, 8, 2, 388, 12], [108, 124, 92, 144, 387, 1118, 600, 8, 2, 388, 12, 1119], [79, 31], [79, 31, 2], [79, 31, 2, 3], [79, 31, 2, 3, 1120], [79, 31, 2, 3, 1120, 2], [79, 31, 2, 3, 1120, 2, 16], [79, 31, 2, 3, 1120, 2, 16, 601], [79, 31, 2, 3, 1120, 2, 16, 601, 93], [79, 31, 2, 3, 1120, 2, 16, 601, 93, 1121], [79, 31, 2, 3, 1120, 2, 16, 601, 93, 1121, 282], [79, 31, 2, 3, 1120, 2, 16, 601, 93, 1121, 282, 389], [79, 31, 2, 3, 1120, 2, 16, 601, 93, 1121, 282, 389, 6], [79, 31, 2, 3, 1120, 2, 16, 601, 93, 1121, 282, 389, 6, 1122], [1, 16], [1, 16, 1123], [1, 16, 1123, 1124], [17, 2], [17, 2, 602], [17, 2, 602, 5], [17, 2, 602, 5, 228], [17, 2, 602, 5, 228, 390], [17, 2, 602, 5, 228, 390, 

In [18]:
# 빈도수 상위 n개 단어 확인

########### fill in!  ############

frequency_dict = {value: key for key, value in tokenizer.word_index.items()}
frequency_dict[576]

'answer'

In [19]:
# 패딩 작업

########### fill in!  ############

max_len = max(len(s) for s in sequences)
print(f"샘플 최대 길이: {max_len}")
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)

샘플 최대 길이: 28
[[   0    0    0 ...    0  108  124]
 [   0    0    0 ...  108  124   92]
 [   0    0    0 ...  124   92  144]
 ...
 [   0    0    0 ...   10 3353  123]
 [   0    0    0 ... 3353  123    2]
 [   0    0    0 ...  123    2 1031]]


In [20]:
# 훈련 데이터에서 레이블 분리 (각 문장의 마지막 단어만 분리해서 y로)

########### fill in!  ############
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]    
print(X)
print(y)

[[   0    0    0 ...    0    0  108]
 [   0    0    0 ...    0  108  124]
 [   0    0    0 ...  108  124   92]
 ...
 [   0    0    0 ...  148   10 3353]
 [   0    0    0 ...   10 3353  123]
 [   0    0    0 ... 3353  123    2]]
[ 124   92  144 ...  123    2 1031]


In [21]:
# 레이블에 대한 one-hot encoding

########### fill in!  ############
y = to_categorical(y, num_classes=vocab_size)
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


2) 모델 설계하기

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [23]:
# LSTM 모델 설계하기

########### fill in!  ############
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = max_len-1))
model.add(LSTM(64))
model.add(Dense(vocab_size, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X,y,epochs = 200, verbose=2)

Epoch 1/200
254/254 - 7s - loss: 7.4841 - accuracy: 0.0284
Epoch 2/200
254/254 - 4s - loss: 6.9722 - accuracy: 0.0268
Epoch 3/200
254/254 - 5s - loss: 6.8544 - accuracy: 0.0292
Epoch 4/200
254/254 - 4s - loss: 6.7650 - accuracy: 0.0382
Epoch 5/200
254/254 - 5s - loss: 6.6758 - accuracy: 0.0419
Epoch 6/200
254/254 - 5s - loss: 6.5816 - accuracy: 0.0465
Epoch 7/200
254/254 - 5s - loss: 6.4863 - accuracy: 0.0476
Epoch 8/200
254/254 - 5s - loss: 6.3994 - accuracy: 0.0488
Epoch 9/200
254/254 - 5s - loss: 6.3165 - accuracy: 0.0502
Epoch 10/200
254/254 - 5s - loss: 6.2368 - accuracy: 0.0539
Epoch 11/200
254/254 - 5s - loss: 6.1615 - accuracy: 0.0546
Epoch 12/200
254/254 - 5s - loss: 6.0870 - accuracy: 0.0615
Epoch 13/200
254/254 - 5s - loss: 6.0102 - accuracy: 0.0631
Epoch 14/200
254/254 - 5s - loss: 5.9361 - accuracy: 0.0678
Epoch 15/200
254/254 - 5s - loss: 5.8586 - accuracy: 0.0688
Epoch 16/200
254/254 - 5s - loss: 5.7828 - accuracy: 0.0703
Epoch 17/200
254/254 - 5s - loss: 5.7089 - accura

In [24]:
# 문장 생성 함수 만들기

def sentence_generation(model, t, current_word, n):
    init_word = current_word
    sentence = ''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen = 5, padding = 'pre')
        result = model.predict_classes(encoded, verbose = 0)
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word 
        sentence = sentence + ' ' + word
    sentence = init_word + sentence
    return sentence


In [26]:
print(sentence_generation(model, tokenizer, 'i', 10))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


i treat treat treat heart dad hope in more germany accuse


In [27]:
print(sentence_generation(model, tokenizer, 'how', 10))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


how treat love president al president about sticker disaster meddling owes


#### 수고하셨습니다 👍